In [1]:
import pertpy as pt


/software/cellgen/team298/ls34/sccoda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings

import decoupler as dc
import scanpy as sc

warnings.filterwarnings("ignore")

## Dataset

In [3]:
adata=sc.read_h5ad('/nfs/team298/ls34/adult_skin/final_adatas/adata_combined_new.h5ad.final.filtered.scrna')
adata


AnnData object with n_obs × n_vars = 1730938 × 32732
    obs: 'sample_id', 'barcode', 'GSE', 'Site_status', 'Patient_status', 'Location', 'Age', 'Sex', 'n_genes', 'dataset_id', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'QC_hi', 'QC_mid', 'lvl5_annotation', 'Mapping_status', 'scanvi_predictions', 'lvl5_annotation_new', 'lvl5_annotation_new2', 'lvl5_annotation_new3', 'lvl5_annotation_new_archive', 'lvl5_annotation_new_preoprhan', 'lvl5_annotation_new10', 'lvl5_annotation_new11', 'test', 'test_n', 'lvl5_annotation_new12', 'lvl5_annotation_new13', 'lvl4_annotation', 'lvl0', 'temp', 't', 'leiden_res0.1', 'Site_status_simple', 'StatusMilo', 'atlas_status', 'atlas_status_reynolds', 'atlas_status_reynolds_simple', 'atlas_status_simple', 'atlas_status_simple2', 'Site_status_binary', 'lvl1_new'
    uns: '

In [4]:
adata.obs["atlas_status_simple"].value_counts()

atlas_status_simple
Eczema_Lesional                                   190291
Psoriasis_Lesional                                145006
Psoriasis_PostRx                                   96485
Eczema_Nonlesional                                 92297
Eczema_PostRx                                      10920
Healthy_Nonlesional_prurigo_calugura_GSE213849      2490
Eczema_Lesional_prurigo_calugura_GSE213849          1400
Name: count, dtype: int64

In [5]:
adata.obs["Site_status_binary"]=adata.obs["Site_status_simple"]
adata.obs.Site_status_binary.value_counts()



Site_status_binary
Nonlesional    1286836
Lesional        336697
PostRx          107405
Name: count, dtype: int64

In [6]:
adata = adata[adata.obs["Site_status_binary"] == "Lesional", :].copy()
adata.obs.Site_status_binary.value_counts()



Site_status_binary
Lesional    336697
Name: count, dtype: int64

In [7]:
adata.obs["Patient"]=adata.obs["sample_id"]
adata.obs["Sample"]=adata.obs["sample_id"]
patient_counts = adata.obs.groupby("Patient").size()
patient_counts


Patient
AD1           4471
AD2           3671
AD3           1740
AD4           2884
AD5           5218
              ... 
SKN8090589    7063
SKN8090591    6446
SKN8090604    4298
SKN8090605    7638
SKN8090607    6061
Length: 63, dtype: int64

In [ ]:
mask = adata.obs["Patient"].map(patient_counts).astype(int) >= 100
adata = adata[mask].copy()


In [ ]:
adata.obs["Group"]=adata.obs["Site_status"]
adata.obs["Major celltype"] = adata.obs["lvl0"]
adata.obs["Cluster"]=adata.obs["lvl5_annotation"] 
adata.obs["disease_overall"]=adata.obs["Patient_status"]


In [ ]:
for x in adata.obs.columns:
    if x not in ["Cluster", "Major celltype", "disease_overall", "Site_status", "Group", "Sample", "Patient"]:
        del(adata.obs[x])
adata


We remove Patient 10 who has had very few sequenced cells.

In [ ]:
adata.layers["counts"]=adata.X.copy()


In [ ]:
adata_i=adata[(adata.obs["Cluster"]=="MigDC (cDC2)")
               ].copy()
adata_i.obs["Cluster"].value_counts()


In [ ]:
ps = pt.tl.PseudobulkSpace()
pdata = ps.compute(
    adata_i, 
    target_col='disease_overall',
    #target_col="disease_overall",
   groups_col="Patient", 
    layer_key="counts", mode="sum", min_cells=100, min_counts=50_000
)
pdata.obs




In [ ]:
pdata.obs.disease_overall.value_counts()

In [ ]:
pdata.obs


In [ ]:
try:
    ps.plot_psbulk_samples(pdata, groupby=["disease_overall", "Major celltype", "Cluster"], figsize=(12, 4)) #"Patient", 
except:
    1

## Axes of variation

In [ ]:
pdata.layers["counts"] = pdata.X.copy()

sc.pp.normalize_total(pdata, target_sum=1e4)
sc.pp.log1p(pdata)
sc.pp.scale(pdata, max_value=10)
sc.tl.pca(pdata)

dc.swap_layer(pdata, "counts", X_layer_key=None, inplace=True)


In [ ]:
adata

In [ ]:
pdata

In [ ]:
# try:
#     dc.get_metadata_associations(
#         pdata,
#         obs_keys=[ "disease_overall", "Patient", "psbulk_n_cells", "psbulk_counts"], # "Treatment", "Patient",
#         obsm_key="X_pca",
#         uns_key="pca_anova",
#         inplace=True,
#     )
#     pdata
# except:
#     1

In [ ]:
1

In [ ]:
# try:
#     dc.plot_associations(
#         pdata,
#         uns_key="pca_anova",
#         obsm_key="X_pca",
#         stat_col="p_adj",
#         obs_annotation_cols=["disease_overall",  #"Treatment", "Efficacy", 
#                              "Major celltype"],
#         titles=["Principle component scores", "Adjusted p-values from ANOVA"],
#     )
# except:
#     1

## Differential expression testing with edgeR

In [ ]:
edgr = pt.tl.PyDESeq2(adata=pdata, design="~disease_overall")


In [ ]:
edgr.fit()


In [ ]:
res_df = edgr.test_contrasts(edgr.contrast(column="disease_overall", baseline="AD", group_to_compare="Psoriasis"))


In [ ]:
edgr.plot_volcano(res_df, 
                  log2fc_thresh=0,
                     to_label=10,
                  #save="fig3/3g.pdf"
                     #    log2fc_thresh=0.
                 )


In [ ]:
fig = edgr.plot_volcano(
    res_df,
    log2fc_thresh=0,
    to_label=10,
    return_fig=True
)

fig.savefig("fig3/3g.pdf", dpi=300, bbox_inches="tight")

In [ ]:
fig = edgr.plot_volcano(
    res_df,
    log2fc_thresh=0,
    to_label=5,
    return_fig=True
)

fig.savefig("fig3/3g_cleaner.pdf", dpi=300, bbox_inches="tight")

In [ ]:
fig = edgr.plot_volcano(
    res_df,
    log2fc_thresh=0,
    to_label=1,
    return_fig=True
)

fig.savefig("fig3/3g_cleanest.pdf", dpi=300, bbox_inches="tight")

In [ ]:
edgr.plot_fold_change(res_df, n_top_vars=20)


In [ ]:
0